In [1]:
import pandas as pd

In [2]:
#import conversion table
conversion1 = pd.read_csv('../utils/conversion_tables/inp_icd_desc_web1.csv')
conversion2 = pd.read_csv('../utils/conversion_tables/servp_icd_desc_web1.csv')
conversion3 = pd.read_csv('../utils/conversion_tables/ICD10_Formatted.csv')

In [ ]:
# keep only the columns we need from the I, S, O tables
def clean_i(df):
    df = df[['ENROLID', 'SEQNUM', 'DISDATE','DX1', 'DX2', 'DX3', 'DX4','DX5', 'DX6', 'DX7', 'DX8', 'DX9','DX10', 'DX11', 'DX12', 'DX13', 'DX14','DX15','PDX']]
    df = df.dropna(subset=['ENROLID'])
    return df
def clean_o(df):
    df = df[['ENROLID', 'SEQNUM', 'DX1', 'DX2', 'DX3', 'DX4', 'SVCDATE']]
    df = df.dropna(subset=['ENROLID'])
    return df
def clean_s(df):
    df = df[['ENROLID', 'SEQNUM', 'DX1', 'DX2', 'DX3', 'DX4', 'PDX', 'DISDATE']]
    df = df.dropna(subset=['ENROLID'])
    return df

In [ ]:

# Combine the I, S, O tables into one dataframe
def CombineI_long(df):
    df_result = pd.melt(df, id_vars=['ENROLID', 'DISDATE'], 
                              value_vars=['DX1', 'DX2', 'DX3', 'DX4', 'DX5', 'DX6', 'DX7', 'DX8', 'DX9', 'DX10', 'DX11', 'DX12', 'DX13', 'DX14', 'DX15','PDX'], 
                              var_name='DX_var', value_name='DX')
    df_result = df_result.drop('DX_var', axis=1)
    return df_result

def CombineO_long(df):
    df_result = pd.melt(df, id_vars=['ENROLID', 'SVCDATE'],
                                value_vars=['DX1', 'DX2', 'DX3', 'DX4'],
                                var_name='DX_var', value_name='DX')
    df_result = df_result.drop('DX_var', axis=1)
    df_result = df_result.rename(columns={'SVCDATE': 'DISDATE'})
    return df_result

def CombineS_long(df):
    df_result = pd.melt(df, id_vars=['ENROLID', 'DISDATE'],
                                value_vars=['DX1', 'DX2', 'DX3', 'DX4', 'PDX'],
                                var_name='DX_var', value_name='DX')
    df_result = df_result.drop('DX_var', axis=1)
    return df_result

# Combine the years of data into one dataframe
def CombineYear(df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11):
    df_all = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11])
    return df_all


In [ ]:
# utilize conversion tables to convert ICD codes

conversion1['ICD9'] = conversion1['ICD9'].str.replace('.', '')
conversion2['ICD9'] = conversion2['ICD9'].str.replace('.', '')
conversion3['ICD9'] = conversion3['ICD9'].str.replace('.', '')
conversion3['search'] = conversion3['search'].str.replace('.', '')

onversion_combined = pd.concat([conversion1, conversion2, conversion3])
conversion_combined = conversion_combined[['search', 'ICD9']]
conversion_combined = conversion_combined.drop_duplicates(subset=['ICD9'], keep='first')
conversion_combined.reset_index(drop=False, inplace=True)
conversion_combined = conversion_combined.drop('index', axis=1)

# Create a dictionary to map ICD-10 codes to ICD-9 codes
icd10_to_icd9 = conversion_combined.set_index('search')['ICD9'].to_dict()
icd10_to_icd9